In [1]:
import sys
sys.path.append('src')

***
## Module Imports

In [2]:
import nltk
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from nltk.metrics.distance import jaccard_distance
from collections.abc import Iterable
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer

In [3]:
from data_utils import load_data
from dimension.lexical import *
from dimension.syntactical import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/victorbadenas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/victorbadenas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /home/victorbadenas/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


***
## Data

In [4]:
train_data, test_data = load_data('data/')
print(
    f"train_data samples: {len(train_data)}, test_data samples: {len(test_data)}"
)

train_data samples: 2234, test_data samples: 3108


In [5]:
train_data.head()

,S1,S2,Gs
0,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...,4.00
1,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...,3.75
2,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl...",2.80
3,"The American Anglican Council, which represent...","The American Anglican Council, which represent...",3.40
4,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....,2.40


In [6]:
test_data.head()

,S1,S2,Gs
0,The problem likely will mean corrective change...,He said the problem needs to be corrected befo...,4.4
1,The technology-laced Nasdaq Composite Index .I...,The broad Standard & Poor's 500 Index .SPX inc...,0.8
2,"""It's a huge black eye,"" said publisher Arthur...","""It's a huge black eye,"" Arthur Sulzberger, th...",3.6
3,SEC Chairman William Donaldson said there is a...,"""I think there's a building confidence that th...",3.4
4,Vivendi shares closed 1.9 percent at 15.80 eur...,"In New York, Vivendi shares were 1.4 percent d...",1.4


***
## Similarity functions

In [7]:
def jaccard_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    return 1 - jaccard_distance(set(s1), set(s2))

In [8]:
def overlap_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return len(intersection) / min(len(s1), len(s2))

In [9]:
def cosine_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return len(intersection) / ((len(s1) * len(s2))**2)

In [10]:
def dice_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return 2 * len(intersection) / (len(s1) + len(s2))

***
## Feature loading

### feature vector builder for dataframe of sentence pairs

Declaration of the function responsible for the iteration over the dataframe containing the sentence pairs (other columns shall be unused). Requires the sentences columns' to be named `"S1"` and `"S2"`.

Returns a numpy array of shape `(n_sentence_pairs, n_features)`

In [11]:
def get_features(df: pd.DataFrame):
    assert "S1" in df.columns, "S1 not in dataframe"
    assert "S2" in df.columns, "S2 not in dataframe"

    features = [None] * len(df)   #preallocated for memory efficiency

    for index, row in df.iterrows():
        sentence1, sentence2 = row['S1'], row['S2']

        # Get all words
        tokenized_1, tokenized_2 = get_tokenized_sentences(
            sentence1, sentence2, return_unique_words=False)
        tokenized_lc_1, tokenized_lc_2 = get_tokenized_sentences_lowercase(
            tokenized_1, tokenized_2, return_unique_words=False)

        # Get words without stopwords
        no_stopwords_1, no_stopwords_2 = filter_stopwords(
            tokenized_1, tokenized_2, return_unique_words=False)
        no_stopwords_lc_1, no_stopwords_lc_2 = filter_stopwords(
            tokenized_lc_1, tokenized_lc_2, return_unique_words=False)

        # Lemmas
        lemmatized_1, lemmatized_2 = get_lemmas(tokenized_1,
                                                tokenized_2,
                                                return_unique_words=False)
        lemmatized_lc_1, lemmatized_lc_2 = get_lemmas(
            tokenized_lc_1, tokenized_lc_2, return_unique_words=False)

        # Name entities
        sentence_ne_1, sentence_ne_2 = get_named_entities(
            tokenized_1, tokenized_2)
        
        #lemmas cleaned from stopwords
        stopwords_and_lemmas1, stopwords_and_lemmas2 = get_lemmas(
            no_stopwords_1, no_stopwords_2, return_unique_words=False)

        stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2 = get_lemmas(
            no_stopwords_lc_1, no_stopwords_lc_2, return_unique_words=False)
        
        # Name entities without stopwords in lowercase
        ne_no_stopwords_1, ne_no_stopwords_2 = filter_stopwords(
            sentence_ne_1, sentence_ne_2, return_unique_words=False, filter_and_return_in_lowercase=True)
        
        # Name entities without stopwords in lowercase and lemmas
        ne_no_stopwords_lemmas_1, ne_no_stopwords_lemmas_2 = get_lemmas( ne_no_stopwords_1, ne_no_stopwords_2, 
                                                                        return_unique_words=False)

        # Bigrams
        bigrams_1, bigrams_2 = get_ngrams(no_stopwords_1, no_stopwords_2, n=2)
        trigrams_1, trigrams_2 = get_ngrams(no_stopwords_1, no_stopwords_2, n=3)
        
        # Bigrams trigrams with sentence tokenizer 
        bigrams_sent_1, bigrams_sent_2 = get_ngrams_with_sent_tokenize(sentence1, sentence2, n=2)
        trigrams_sent_1, trigrams_sent_2 = get_ngrams_with_sent_tokenize(sentence1, sentence2, n=3)
        
        # Lesk
        lesk_1, lesk_2 = get_lesk_sentences(tokenized_1, tokenized_2)
        lesk_lc_1, lesk_lc_2 = get_lesk_sentences(tokenized_lc_1, tokenized_lc_2)


        # Stemmer
        stemmed_1, stemmed_2 = get_stemmed_sentences(sentence1, sentence2)
        
        
        # Synset
        average_path = get_synset_similarity(tokenized_1, tokenized_2, "path")
        average_lch = get_synset_similarity(tokenized_1, tokenized_2, "lch")
        average_wup = get_synset_similarity(tokenized_1, tokenized_2, "wup")
        average_lin = get_synset_similarity(tokenized_1, tokenized_2, "lin")
        
        average_lc_path = get_synset_similarity(tokenized_lc_1, tokenized_lc_2, "path")
        average_lc_lch = get_synset_similarity(tokenized_lc_1, tokenized_lc_2, "lch")
        average_lc_wup = get_synset_similarity(tokenized_lc_1, tokenized_lc_2, "wup")
        average_lc_lin = get_synset_similarity(tokenized_lc_1, tokenized_lc_2, "lin")
        
        
        # ALL Features
        features[index] = [
            jaccard_similarity(tokenized_1, tokenized_2),
            # jaccard_similarity(tokenized_lc_1, tokenized_lc_2),
            # jaccard_similarity(no_stopwords_1, no_stopwords_2),
            jaccard_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
            # jaccard_similarity(lemmatized_1, lemmatized_2),
            # jaccard_similarity(lemmatized_lc_1, lemmatized_lc_2),
            jaccard_similarity(sentence_ne_1, sentence_ne_2),
            jaccard_similarity(stopwords_and_lemmas1, stopwords_and_lemmas2),
            # jaccard_similarity(stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2),
            # jaccard_similarity(bigrams_1, bigrams_2),
            jaccard_similarity(trigrams_1, trigrams_2),
            # jaccard_similarity(bigrams_sent_1, bigrams_sent_2),
            jaccard_similarity(trigrams_sent_1, trigrams_sent_2),
            jaccard_similarity(lesk_1, lesk_2),
            jaccard_similarity(lesk_lc_1, lesk_lc_2),
            jaccard_similarity(stemmed_1, stemmed_2),
            dice_similarity(tokenized_1, tokenized_2),
            # dice_similarity(tokenized_lc_1, tokenized_lc_2),
            dice_similarity(no_stopwords_1, no_stopwords_2),
            dice_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
            # dice_similarity(lemmatized_1, lemmatized_2),
            # dice_similarity(lemmatized_lc_1, lemmatized_lc_2),
            # dice_similarity(sentence_ne_1, sentence_ne_2),
            # dice_similarity(stopwords_and_lemmas1, stopwords_and_lemmas2),
            dice_similarity(stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2),
            dice_similarity(bigrams_1, bigrams_2),
            # dice_similarity(trigrams_1, trigrams_2),
            # dice_similarity(bigrams_sent_1, bigrams_sent_2),
            # dice_similarity(trigrams_sent_1, trigrams_sent_2),
            # dice_similarity(lesk_1, lesk_2),
            # dice_similarity(lesk_lc_1, lesk_lc_2),
            # dice_similarity(stemmed_1, stemmed_2),
            # average_path,
            # average_lch,
            # average_wup,
            average_lin,
            # average_lc_path,
            average_lc_lch,
            average_lc_wup,
            average_lc_lin
        ]
        # BEST Features selection 
        """features[index] = [
            jaccard_similarity(tokenized_1, tokenized_2),
            jaccard_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
            jaccard_similarity(sentence_ne_1, sentence_ne_2),
            jaccard_similarity(stopwords_and_lemmas1, stopwords_and_lemmas2),
            jaccard_similarity(trigrams_1, trigrams_2),
            jaccard_similarity(trigrams_sent_1, trigrams_sent_2),
            jaccard_similarity(lesk_1, lesk_2),
            jaccard_similarity(lesk_lc_1, lesk_lc_2),
            jaccard_similarity(stemmed_1, stemmed_2),
            dice_similarity(tokenized_1, tokenized_2),
            dice_similarity(no_stopwords_1, no_stopwords_2),
            dice_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
            dice_similarity(stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2),
            dice_similarity(bigrams_1, bigrams_2),
            average_lin,
            average_lc_lch,
            average_lc_wup,
            average_lc_lin
        ]"""
    return np.array(features)

In [12]:
# TEST CELL
def show_sentences_with_applications(train_head, results):
    for index, row in train_head.iterrows():
        sentence1, sentence2 = row['S1'], row['S2']
        gs = row['Gs']
        print(f"-{index}- ({gs})")
        print(sentence1)
        print(sentence2)
        if(results is not None):
            for result in results:
                print("-----------------------------")
                print(result[index][0])
                print(result[index][1])
        print("********************************************************")
        
def apply_function(data, method_to_apply, params = None):
    results = []
    for sentence1, sentence2 in data:
        if params is None:
            result1, result2 = method_to_apply(sentence1, sentence2)
        elif len(params) == 1:
            result1, result2 = method_to_apply(sentence1, sentence2, params[0])
        elif len(params) == 2:
            result1, result2 = method_to_apply(sentence1, sentence2, params[0], params[1])  
        results.append((result1, result2))
    return results


train_head = train_data.head()
sentence_list = [(row['S1'], row['S2']) for index, row in train_head.iterrows()]

results = []
tokens = apply_function(sentence_list, get_tokenized_sentences)
lesk = apply_function(tokens, get_lesk_sentences)
results.append(lesk)
show_sentences_with_applications(train_head, results)

-0- (4.0)
But other sources close to the sale said Vivendi was keeping the door open to further bids and hoped to see bidders interested in individual assets team up.
But other sources close to the sale said Vivendi was keeping the door open for further bids in the next day or two.
-----------------------------
['But', 'other.a.01', 'reservoir.n.04', 'near.r.01', 'to', 'the', 'sale.n.05', 'state.v.01', 'Vivendi', 'be.v.12', 'retain.v.02', 'the', 'doorway.n.01', 'overt.a.01', 'to', 'further', 'bid.n.04', 'and', 'hope.v.01', 'to', 'visualize.v.01', 'bidder.n.02', 'interested.a.01', 'in', 'individual.a.01', 'assets.n.01', 'team.v.01', 'up.r.03', '.']
['But', 'other.a.01', 'reservoir.n.04', 'near.r.01', 'to', 'the', 'sale.n.05', 'pronounce.v.01', 'Vivendi', 'be.v.12', 'retain.v.02', 'the', 'doorway.n.01', 'receptive.a.02', 'for', 'further', 'command.n.01', 'in', 'the', 'next', 'sidereal_day.n.01', 'or', 'two', '.']
********************************************************
-1- (3.75)
Micron 

In [13]:
# TEST cell don't delete it =D

first = "My Bonnie White lies over the ocean, in Picadilli Circus at 3:00pm."
second = "My Bonnie lied over the sea! Over the sea..."

tokenized_1, tokenized_2 = get_tokenized_sentences(first,
                                                   second,
                                                   return_unique_words=False)
no_stopwords_1, no_stopwords_2 = filter_stopwords(tokenized_1, tokenized_2, return_unique_words=False)
sentence_ne_1, sentence_ne_2 = get_named_entities(no_stopwords_1, no_stopwords_2)

print(tokenized_1)
print(tokenized_2)
print(sentence_ne_1)
print(sentence_ne_2)

average_sim = get_synset_similarity(tokenized_1, tokenized_2, "path")
print(average_sim)
average_sim = get_synset_similarity(tokenized_1, tokenized_2, "lch")
print(average_sim)
average_sim = get_synset_similarity(tokenized_1, tokenized_2, "wup")
print(average_sim)
average_sim = get_synset_similarity(tokenized_1, tokenized_2, "lin")
print(average_sim)

['My', 'Bonnie', 'White', 'lies', 'over', 'the', 'ocean', ',', 'in', 'Picadilli', 'Circus', 'at', '3:00pm', '.']
['My', 'Bonnie', 'lied', 'over', 'the', 'sea', '!', 'Over', 'the', 'sea', '...']
['My', 'Bonnie White', 'lies', 'ocean', 'Picadilli Circus', '3:00pm']
['My', 'Bonnie', 'lied', 'sea', 'Over', 'sea', '...']
0.19868326118326116
0.5725251800393162
0.34814814814814815
0.3526340967251205


In [14]:
# TEST CELL
def show_sentences_with_applications(train_head, results):
    for index, row in train_head.iterrows():
        sentence1, sentence2 = row['S1'], row['S2']
        gs = row['Gs']
        print(f"-{index}- ({gs})")
        print(sentence1)
        print(sentence2)
        if(results is not None):
            for result in results:
                print("-----------------------------")
                print(result[index][0])
                print(result[index][1])
        print("********************************************************")
        
def apply_function(data, method_to_apply, params = None):
    results = []
    for sentence1, sentence2 in data:
        if params is None:
            result1, result2 = method_to_apply(sentence1, sentence2)
        elif len(params) == 1:
            result1, result2 = method_to_apply(sentence1, sentence2, params[0])
        elif len(params) == 2:
            result1, result2 = method_to_apply(sentence1, sentence2, params[0], params[1])  
        results.append((result1, result2))
    return results


train_head = train_data.head()
sentence_list = [(row['S1'], row['S2']) for index, row in train_head.iterrows()]

results = []
tokens = apply_function(sentence_list, get_tokenized_sentences)
results.append(tokens)

show_sentences_with_applications(train_head, results)

-0- (4.0)
But other sources close to the sale said Vivendi was keeping the door open to further bids and hoped to see bidders interested in individual assets team up.
But other sources close to the sale said Vivendi was keeping the door open for further bids in the next day or two.
-----------------------------
['But', 'other', 'sources', 'close', 'to', 'the', 'sale', 'said', 'Vivendi', 'was', 'keeping', 'the', 'door', 'open', 'to', 'further', 'bids', 'and', 'hoped', 'to', 'see', 'bidders', 'interested', 'in', 'individual', 'assets', 'team', 'up', '.']
['But', 'other', 'sources', 'close', 'to', 'the', 'sale', 'said', 'Vivendi', 'was', 'keeping', 'the', 'door', 'open', 'for', 'further', 'bids', 'in', 'the', 'next', 'day', 'or', 'two', '.']
********************************************************
-1- (3.75)
Micron has declared its first quarterly profit for three years.
Micron's numbers also marked the first quarterly profit in three years for the DRAM manufacturer.
---------------------

In [15]:
# TEST cell don't delete it =D

first = "My Bonnie White lies over the ocean, in Picadilli Circus at 3:00pm."
second = "My Bonnie lied over the sea! Over the sea..."

tokenized_1, tokenized_2 = get_tokenized_sentences(first,
                                                   second,
                                                   return_unique_words=False)
no_stopwords_1, no_stopwords_2 = filter_stopwords(tokenized_1, tokenized_2, return_unique_words=False)
sentence_ne_1, sentence_ne_2 = get_named_entities(no_stopwords_1, no_stopwords_2)

print(tokenized_1)
print(tokenized_2)
print(sentence_ne_1)
print(sentence_ne_2)
#TEST cell

['My', 'Bonnie', 'White', 'lies', 'over', 'the', 'ocean', ',', 'in', 'Picadilli', 'Circus', 'at', '3:00pm', '.']
['My', 'Bonnie', 'lied', 'over', 'the', 'sea', '!', 'Over', 'the', 'sea', '...']
['My', 'Bonnie White', 'lies', 'ocean', 'Picadilli Circus', '3:00pm']
['My', 'Bonnie', 'lied', 'sea', 'Over', 'sea', '...']


### Features extraction

Using the function declared above, the features are extracted from the `train_data` dataframe. Also the Gold Standard is extracted from its column in the dataframe. The shapes for both numpy vectors are displayed. 

In [16]:
train_features = get_features(train_data)
train_gs = train_data['Gs'].to_numpy()
print(f"train_features.shape: {train_features.shape}")
print(f"train_gs.shape: {train_gs.shape}")

train_features.shape: (2234, 18)
train_gs.shape: (2234,)


In [17]:
test_features = get_features(test_data)
test_gs = test_data['Gs'].to_numpy()
print(f"train_features.shape: {test_features.shape}")
print(f"train_gs.shape: {test_gs.shape}")

train_features.shape: (3108, 18)
train_gs.shape: (3108,)


### Feature scaling

features are scaled using sklearns StandardScaler, where the mean is substracted for each feature and it's divided by the variance of the feature to obtain a unified feature space with zero mean and unit variance.

In [18]:
scaler = StandardScaler()
scaler.fit(train_features)
train_features_scaled = scaler.transform(train_features)
test_features_scaled = scaler.transform(test_features)

### Split definition for GridSearch

In [19]:
all_data = np.concatenate([train_features_scaled, test_features_scaled])
all_labels = np.concatenate([train_gs, test_gs])
test_fold = np.array([-1]*train_features_scaled.shape[0] + [0]*test_features_scaled.shape[0])
print(all_data.shape, test_fold.shape)
ps = PredefinedSplit(test_fold)

(5342, 18) (5342,)


### SVR Training

In [20]:
pearson_scorer = make_scorer(lambda y, y_hat: pearsonr(y, y_hat)[0])

gammas = np.logspace(-6, -1, 6)
Cs = np.array([0.5, 1, 2, 4, 8, 10, 15, 20, 50, 100, 200, 375, 500, 1000])
epsilons = np.linspace(0.1, 1, 10)
param = dict(gamma=gammas, C=Cs, epsilon=epsilons)

svr = SVR(kernel='rbf', tol=1)
gssvr = GridSearchCV(svr,
                     param,
                     cv=ps,
                     scoring=pearson_scorer,
                     n_jobs=-1,
                     verbose=1)
gssvr = gssvr.fit(all_data, all_labels)

Fitting 1 folds for each of 840 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 684 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed:   27.1s finished


In [21]:
best_parameters = gssvr.best_params_
best_model = SVR(kernel='rbf', tol=1, **best_parameters)
train_predictions = best_model.fit(train_features_scaled, train_gs).predict(train_features_scaled)

### Test Inference

In [22]:
test_predictions = best_model.predict(test_features_scaled)

### Evaluation metrics

In [23]:
train_correlation = pearsonr(train_predictions, train_gs)[0]
test_correlation = pearsonr(test_predictions, test_gs)[0]

In [24]:
print('Train pearsonr: ', train_correlation)
print('Test pearsonr: ', test_correlation)
print('The best value of gamma:', gssvr.best_estimator_.gamma)
print('The best value of C:', gssvr.best_estimator_.C)
print('The best value of epsilon:', gssvr.best_estimator_.epsilon)

Train pearsonr:  0.8612974649511776
Test pearsonr:  0.7456997951303734
The best value of gamma: 0.1
The best value of C: 1.0
The best value of epsilon: 0.4


***

# Do not delete ;)

## Recorded Results
|SVR|jaccard|overlap|cosine|dice|jaccard+dice|no_stop_lc_dice|no_stop_lemmas_dice|no_stop_lemmas_lc_dice|forward_search_12_feats|mutual_info_sel_13_feats|pca_12_feats|forward_search_pca_6_feats|fs_pca_12_features|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|train correlation|0.6402|0.6233|0.3732|0.6397|0.6379|0.6017|0.6465|0.649|0.6544|0.6364|0.6454|0.6669|0.6485|
|test_correlation|0.6208|0.6198|0.1056|0.6483|0.6087|0.5905|0.6634|0.652|0.6843|0.6428|0.665|0.6755|0.6848|
|C|10.0|500.0|100.0|375.0|1000.0|2.0|100.0|500.0|15.0|20.0|500.0|20.0|2.0|
|gamma|0.01|0.01|0.1|0.0001|0.001|0.1|0.01|1e-5|0.01|0.001|0.001|0.01|0.01|
|epsilon|0.9|1.0|1.0|0.9|0.2|0.1|0.9|0.1|0.8|0.3|1.0|0.8|0.4|